# 환경세팅

In [2]:
# 📌
!pip install --upgrade openai
!pip install langchain
!pip install langchain_openai
!pip install rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 2.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.5 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 6.3 MB/s eta 0:00:00:00:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=78e0072d9fd3d3087de255a7d653a8dbddaee3bb4b7d26fdd9aa47b0b3279aad
  Stored in directory: /home/jovyan/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


# 라이브러리

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
#템플릿 생성 - 채팅 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
import getpass
import os
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

import pandas as pd
from rouge_score import rouge_scorer
import re

In [2]:
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4o")

 ········


## 지연/파업 템플릿

In [4]:
template=pd.read_csv("template.csv")
delay_template=template.loc[0,"template"]
strike_template=template.loc[1,"template"]

In [4]:
시간표변경 합칠때 쓰세용
pd.concat([template,시간표변경.csv]).to_csv("template.csv")

SyntaxError: invalid syntax (3947959249.py, line 1)

# Text_generation 함수

In [32]:
source_content="""
벌초하던 낫 꺼내 들고 지하철서 난동 부린 80대男
입력2024.06.05. 오후 7:00
종각역서 흉기 난동 신고접수
경찰, 특수협박 혐의로 체포
벌초 다녀오다 술마신뒤 범행



지하철 1호선 종각역에서 80대 노인이 낫을 들고 난동을 부린다는 신고가 접수돼 경찰이 체포했다.

5일 경찰과 서울교통공사 등에 따르면 이날 오후 6시 20분께 서울 지하철 1호선 종각역 역사에서 흉기(낫)를 소지하고 있던 80대 남성 A 씨를 특수 협박 혐의로 체포했다.

경찰에 따르면 이날 오후 6시 10분께 경찰에 ‘한 남성이 열차 안에서 난동을 부리고 있다’는 신고가 접수됐다. 남성은 벌초를 다녀오다 술을 마신 후 낫을 열차 안에서 꺼내든 것으로 알려졌다.

난동으로 시민 1명이 찰과상 부상을 입었다는 피해 사항이 접수된 것으로 전해졌다.

경찰 출동 당시 열차 승무원이 이미 A 씨의 흉기를 빼앗은 상태였다. 소란으로 1호선 열차도 약 6분간 정차했다. 서울교통공사 관계자는 “신고를 받고 승무원이 출동해 흉기를 회수한 뒤 종각역에서 A 씨와 함께 하차해 경찰에 인계했다”고 밝혔다.

경찰 관계자는 “자세한 사건 경위와 피해 상황에 대해 조사할 방침”이라고 밝혔다. 역 관계자들은 폐쇄회로(CC)TV 화면을 돌려보면서 상황을 파악하고 있다.

"""
category="delay"

In [ ]:
memory.clear()

In [66]:
#지연
if category=="delay":
    prompt_template=delay_template
    chain_lst=["지연/사고 일시","지연/사고 노선","지연/사고 이유"]
#파업
elif category=="strike":
    prompt_template=strike_template
    chain_lst=["파업 일시","파업 노선","파업 이유"]
#시간표변경
else:
    pass
prompt = ChatPromptTemplate(
template=prompt_template,
messages=[
SystemMessagePromptTemplate.from_template(
"""Objective:
Learn subway information through chaining and use that information to write a coherent and informative blog post using a template.
Basic Setting
Your name is Jitong.
You are a subway information supporter and a Naver Blog Power Blogger.
You are well-informed about information and value communication with readers.
Your goal is to help subway users by providing accurate and practical information for convenient and safe subway use.

Features and Activities
You read the latest news related to subway """+ category+""", and write blog posts to accurately deliver these in a readable and high-quality manner to readers.
You alleviate readers' inconveniences and speak empathetically through the blog, understanding the sentiments of subway users.

Communication Style
You use professional yet warm and easy-to-understand language.
You emphasize the ability to explain things in a way that is accessible to all age groups.
Your blog posts should end with the forms -어요, -이에요/예요, -(이)여요, -(이)요.

Hallucination
You always generate blog posts based on verifiable factual statements.
You speak mainly about factual information related to subways and do not add information about subways on your own.
"""
),
# The `variable_name` here is what must align with memory
MessagesPlaceholder(variable_name="chat_history"),
HumanMessagePromptTemplate.from_template("{question}")
]
)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
#정보학습
conversation({"question":"subway strike information(article) :"+source_content+ " Just REVIEW subway " +category+ " information"})
#정보학습 - 체이닝 : chain_lst -> 일시, 노선, 이유
for i in chain_lst:
    r=conversation({"question":"REVIEW ARTICLE, Using the provided information \n write "+i+":"})
#글작성 - 체이닝 -> 정확성
conversation({"question":"블로그 글 작성해줘"})
conversation({"question":"1. 뉴스기사의 내용을 학습해 2. 학습한 뉴스기사와 블로그글을 비교해 3.블로그 글에 틀린 정보가 있다면 수정해 뉴스기사: "+source_content+"블로그 글 :"})
#글 작성 - 템플릿
result=conversation({"question":prompt_template})




> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
Learn subway information through chaining and use that information to write a coherent and informative blog post using a template.
Basic Setting
Your name is Jitong.
You are a subway information supporter and a Naver Blog Power Blogger.
You are well-informed about information and value communication with readers.
Your goal is to help subway users by providing accurate and practical information for convenient and safe subway use.

Features and Activities
You read the latest news related to subway delay, and write blog posts to accurately deliver these in a readable and high-quality manner to readers.
You alleviate readers' inconveniences and speak empathetically through the blog, understanding the sentiments of subway users.

Communication Style
You use professional yet warm and easy-to-understand language.
You emphasize the ability to explain things in a way that is accessible to all age groups.
Your blog po

In [67]:
print(result["text"])

### 위험천만! 80대 남성의 지하철 시끌 사건

안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

#### 지연/사고 일시
2024년 6월 5일 오후 7시

#### 지연/사고 노선
2호선

#### 지연/사고 이유
80대 남성의 흉기 난동

#### 문의 사항 링크
[지하철 온다 문의하기](https://blog.naver.com/subway__onda/223258646349)

---

2024년 6월 5일 오후 7시, 서울 지하철 2호선에서 큰 사건이 발생했어요. 80대 남성이 흉기를 휘두르며 소란을 일으켜 많은 승객들이 공포에 떨었고, 이로 인해 지하철 운행이 한동안 중단되었어요. 다행히 경찰의 빠른 출동으로 큰 인명 피해는 없었지만, 이로 인해 많은 시민들이 큰 불편을 겪었답니다.

### 어떻게 대처해야 할까요?

이런 예기치 못한 상황에서 어떻게 대처해야 할지 몰라 당황스러운 분들을 위해 몇 가지 팁을 준비했어요.

#### 1. 침착하게 대응하기
긴급 상황에서는 무엇보다 침착함이 중요해요. 주변 사람들과 협력하여 비상구를 찾고, 안전한 장소로 이동하세요.

#### 2. 안내 방송 듣기
지하철 내 안내 방송은 중요한 정보를 제공해요. 지연 사유와 대처 방법 등을 안내해 주니 꼭 귀 기울여 들어주세요.

#### 3. 지하철 앱 활용하기
지하철 관련 앱을 통해 실시간 정보를 확인하고, 대체 교통 수단을 미리 계획해 두는 것도 좋은 방법이에요. '지하철 온다' 앱을 통해 실시간 정보를 쉽게 확인할 수 있어요.

### 여러분의 경험을 들려주세요

이번 사건으로 많은 분들이 큰 불편을 겪으셨을 텐데요, 여러분의 소중한 경험을 댓글로 남겨주세요. 여러분의 의견이 모여 더 나은 지하철 환경을 만드는 데 큰 도움이 될 거예요.

---

지하철 지연으로 아침마다 출근하기 힘드네요. 오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터

In [50]:
print(result)

### {{블로그 제목}}
"80세 할아버지의 지하철 소동! 안전하게 대처하는 방법 알아보기"

### {{시작하는 말}}
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

### {{지연/사고 일시}}
2024년 6월 5일

### {{지연/사고 노선}}
지하철 2호선

### {{지연/사고 이유}}
80세 할아버지가 낫을 휘두르는 소동 발생

### {{문의 사항 링크}}
[문의 사항 및 자세한 내용 보기](https://www.subwaynews.com/incident/20240605)

### {{뉴스기사를 이용해 블로그로 작성한 글}}
지하철을 이용하다 보면 예기치 못한 일들이 발생하곤 해요. 최근 2024년 6월 5일에 있었던 사건도 그 중 하나였어요. 이날 오전, 지하철 2호선에서 80세 할아버지가 낫을 휘두르는 소동이 발생했어요. 할아버지는 정신 건강 문제가 있는 상태로, 원래는 잡초를 제거하기 위해 낫을 사용하고 있었는데, 그 상태로 지하철에 탑승하면서 문제가 발생한 거죠.

이 사건으로 인해 지하철이 즉시 멈추고, 경찰과 응급 대응팀이 신속하게 출동했어요. 다행히도 큰 부상자는 없었지만, 많은 승객들이 놀랐을 거예요. 현재 할아버지는 경찰의 보호 아래 있으며, 추가적인 조사가 진행 중이에요.

이런 상황이 발생했을 때, 여러분은 어떻게 대처해야 할까요? 먼저, 실시간 정보를 확인하는 것이 중요해요. '지하철 온다' 앱을 통해 지연 정보를 빠르게 확인하고, 다른 대체 교통수단을 찾아보는 것이 좋겠어요. 또한, 이런 상황에서는 침착하게 주변 사람들과 협력하는 것이 중요해요. 서로를 배려하면서 안전하게 대처할 수 있도록 노력해야겠죠?

### {{마무리 말}}
오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기
https:

### 템플릿 평가

### 📌 Metric 코드 - 키워드 & 요약문 버전 합침

In [35]:
def evaluate_blog_metric(title, date, article, category, blog_post):

    # Combine title, date, and article into a single news string
    news = f"""<뉴스 제목>: {title},
    <뉴스 생성일자>: {date},
    <뉴스 본문>: {article}"""

    # Determine keywords based on category
    if category == "delay":
        keywords = ["지연/사고 일시", "지연/사고 노선", "지연/사고 이유"]
    elif category == "strike":
        keywords = ["파업 일시", "파업 노선", "파업 이유"]
    else:
        keywords = ["시간표 변경 일시", "변경된 시간표", "변경 이유"]

    # Calculate the keyword-based metric
    matches = sum(1 for keyword in keywords if keyword in blog_post)
    rouge_keyword = matches / len(keywords)

    # Generate summaries for reference and generated content
    summarizer_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are talented at summarizing text without missing any important information."),
        ("user", "Read this article and summarize in 3-5 sentences ONLY in Korean. Article: {article}, Summary :")
    ])

    summarize_chain = summarizer_prompt | llm | output_parser
    reference = summarize_chain.invoke({"article": news})
    generated = summarize_chain.invoke({"article": blog_post})

    # Calculate the ROUGE score for summaries
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    rouge_summary = scorer.score(reference, generated)

    return rouge_keyword, rouge_summary['rouge1'].recall

### 📌 QGQA 코드

In [36]:
# QGQA를 위한 프롬프트 정의
# PROMPT1 - 뉴스 기반 질문 생성
qgqa_prompt_1 = ChatPromptTemplate.from_messages([
    ("system", """You are an AI trained to generate insightful questions from a given article."""),
    ("user", """
    1. Read the article about current Seoul subway news.
    2. Find out what is the main incident related with subway in the article.
    3. Find the date that this article was issued
    4. Find every date-related expression in the article.
    5. Compare 3 and 4, and find out the date the incident was occured.
    6. Find every expression related to line of subway in the article.
    7. Find out which line the incident is about.
    8. Create 2 questions to identify the main points of the news (date of the incident, subway line of the incident). The questions should be 5-way multiple choice questions where you have to choose one of the choices from 1 to 5. One of the options must be “Unknown” and the selection for the “Date of Event” question must be in the format 2018년 3월 18일. Hint is provided with every questions

    <Example Question>: 'Where did the subway incident occur? (1) Gangnam Station (2) Seongsu Station (3) Suyu Station (4) Ankguk Station (5) Unknown)',
    <Example Hint>:''
    News Article= {input}
    """),
    ("system", """Generate 2 questions. All should be in Korean ONLY.

Template:
Question1: {{}},
Hint1: {{}},
Question2: {{}},
Hint2: {{}}""")
])
qgqa_chain_1 = qgqa_prompt_1 | llm | output_parser


# PROMPT2 - 뉴스에 대해 답변 도출
qgqa_prompt_2 = ChatPromptTemplate.from_messages([
    ("system", """You read a news article and answer a question accurately based on what you read."""), # 페르소나 부여
    ("user", """
    You read a news article like this:
    1. Read the article about current Seoul subway news.
    2. Find out what is the main incident related with subway in the article.
    3. Find the date that this article was issued
    4. Find every date-related expression in the article.
    5. Compare 3 and 4, and find out the date the incident was occured.
    6. Find every expression related to line of subway in the article.
    7. Find out which line the incident is about.
    Then you answer a question accurately based on what you read.
    Example= “1번, 5번, 4번”,
    news= {input}
    Questions= {question}
    """),
    ("system", """"Template(MUST FOLLOW): Answer1: {{answer_1}}번, Answer2: {{answer_2}}번""")
])
qgqa_chain_2 = qgqa_prompt_2 | llm | output_parser


# PROMPT3 입력 - 블로그글에 대해 답변 도출
qgqa_prompt_3 = ChatPromptTemplate.from_messages([
    ("system", """You read a blog article and answer a question accurately based on what you read."""), # 페르소나 부여
    ("user", """
    You read a blog article like this:
    1. Read the blog about current Seoul subway news.
    2. Find out what is the main incident related with subway in the article.
    3. Find the date that this article was issued
    4. Find every date-related expression in the article.
    5. Compare 3 and 4, and find out the date the incident was occured.
    6. Find every expression related to line of subway in the article.
    7. Find out which line the incident is about.
    Then you answer a question accurately based on what you read.
    Example= “1번, 5번, 4번”,
    news= {input}
    Questions= {question}
    """),
    ("system", """"Template(MUST FOLLOW): Answer1: {{answer_1}}번, Answer2: {{answer_2}}번""")
])
qgqa_chain_3 = qgqa_prompt_3 | llm | output_parser


# evaluate_blog_qgqa 함수 정의
def evaluate_blog_qgqa(title, date, article, blog_post):

  # news = 제목 + 날짜 + 본문
  news = f"""<뉴스 제목>: {title},
  <뉴스 생성일자>: {date},
  <뉴스 본문>: {article}"""

  # qgqa
  question_list = qgqa_chain_1.invoke({"input": news})
  answer_list_news = qgqa_chain_2.invoke({"input": news, "question": question_list})
  answer_list_blog = qgqa_chain_3.invoke({"input": blog_post, "question": question_list})
  return answer_list_news == answer_list_blog, question_list, answer_list_news, answer_list_blog

### 진규오빠 G-EVAL 코드

In [37]:
df=pd.read_csv("GEVAL.csv",index_col="Unnamed: 0")
eval_dic = {'Cosistency': df.iloc[0,0], 'Human_Likeness':df.iloc[1,0], 'Coherence':df.iloc[1,0], 'Blog':df.iloc[1,0], 'Fluency':df.iloc[1,0]}
eval=[df.iloc[0,0],df.iloc[1,0],df.iloc[2,0],df.iloc[3,0],df.iloc[4,0]]

In [38]:
def evaluate_blog(blog_post, evaluation_prompt, source=None):

    evaluation_result = ""

    # 정확성 이외 평가 (source 필요 없음)
    if source==None:
        prompt = ChatPromptTemplate.from_messages([
        ("system", evaluation_prompt),
        ("user", "{input}")
        ])
        chain=prompt|llm|output_parser
        evaluation_result = chain.invoke({"input": f"blog content: {blog_post}"})


    # 정확성 평가 (source 필요)
    if source is not None:
        prompt = ChatPromptTemplate.from_messages([
        ("system", evaluation_prompt),
        ("user", "{input}")
        ])
        chain = prompt | llm | output_parser
        evaluation_result=chain.invoke({"input": "blog content:"+blog_post+"article, source:"+source})
    return evaluation_result

## 📌 평가지표 코드 병합 및 적용

In [39]:
# 최종 평가 함수
def evaluation_total(title, date, article, category, blog_post):
    # news = 제목 + 날짜 + 본문
    news = f"""<뉴스 제목>: {title},
   <뉴스 생성일자>: {date},
   <뉴스 본문>: {article}"""

    # Step 1(메트릭)
    metric_result = evaluate_blog_metric(title, date, article, category, blog_post)
    if not metric_result[0] == 1:
        return f'Fail(Metric keyword: {metric_result[0]})'

    #if metric_result[1] < 0.2:
    #    return f'Fail(Metric summary: {metric_result[0]})'

    # Step 2(QGQA)
    qgqa_result = evaluate_blog_qgqa(title, date, article, blog_post)
    if qgqa_result[0] != True:
        return f'''Fail(QGQA: {qgqa_result[1]},
        {qgqa_result[2]},
         {qgqa_result[3]})'''

    # Step 3(G-EVAL)
    scores = []
    for i in list(eval_dic.items()):
        if i[0] == 'Cosistency':
            
            score = evaluate_blog(blog_post, i[1], source=news)
            pattern = r"Scores\(SCORE ONLY\): (\d+)"
            match = re.search(pattern, score)
            if match:
                consistency_score = int(match.group(1))
                scores.append(consistency_score)
            else:
                return "No Consistency score found."
        else:
            score = evaluate_blog(blog_post, i[1])
            pattern = r"Scores \(SCORE ONLY\): (\d+)"
            match = re.search(pattern, score)
            if match:
                scores_score = float(match.group(1))
                scores.append(scores_score)
            else:
                return "No Scores (SCORE ONLY) score found."

        if scores[0] <= 4:
            return 'Fail(G-EVAL: Accuracy)'
        if not all(score >= 3 for score in scores[1:5]):
            return 'Fail(G-EVAL: etc)'

    return True # 모든 조건 통과시 True

In [40]:
# 최종 함수(evaluation_total) 적용 예시
source_content="""
벌초하던 낫 꺼내 들고 지하철서 난동 부린 80대男
입력2024.06.05. 오후 7:00
"""
title = "벌초하던 낫 꺼내 들고 지하철서 난동 부린 80대男"
date = '2024.06.05. 19:00'
article = """종각역서 흉기 난동 신고접수
경찰, 특수협박 혐의로 체포
벌초 다녀오다 술마신뒤 범행



지하철 1호선 종각역에서 80대 노인이 낫을 들고 난동을 부린다는 신고가 접수돼 경찰이 체포했다.

5일 경찰과 서울교통공사 등에 따르면 이날 오후 6시 20분께 서울 지하철 1호선 종각역 역사에서 흉기(낫)를 소지하고 있던 80대 남성 A 씨를 특수 협박 혐의로 체포했다.

경찰에 따르면 이날 오후 6시 10분께 경찰에 ‘한 남성이 열차 안에서 난동을 부리고 있다’는 신고가 접수됐다. 남성은 벌초를 다녀오다 술을 마신 후 낫을 열차 안에서 꺼내든 것으로 알려졌다.

난동으로 시민 1명이 찰과상 부상을 입었다는 피해 사항이 접수된 것으로 전해졌다.

경찰 출동 당시 열차 승무원이 이미 A 씨의 흉기를 빼앗은 상태였다. 소란으로 1호선 열차도 약 6분간 정차했다. 서울교통공사 관계자는 “신고를 받고 승무원이 출동해 흉기를 회수한 뒤 종각역에서 A 씨와 함께 하차해 경찰에 인계했다”고 밝혔다.

경찰 관계자는 “자세한 사건 경위와 피해 상황에 대해 조사할 방침”이라고 밝혔다. 역 관계자들은 폐쇄회로(CC)TV 화면을 돌려보면서 상황을 파악하고 있다.


""" # 본문
category = 'delay'
blog_post = result

In [76]:
print(evaluation_total(title, date, article, category, blog_post))

Fail(QGQA: Question1: 사건이 발생한 날짜는 언제입니까? (1) 2024년 6월 3일 (2) 2024년 6월 4일 (3) 2024년 6월 5일 (4) 2024년 6월 6일 (5) 알 수 없음
Hint1: 뉴스 기사 생성일자와 사건 발생일자를 비교해보세요.

Question2: 사건이 발생한 지하철 노선은 무엇입니까? (1) 1호선 (2) 2호선 (3) 3호선 (4) 4호선 (5) 알 수 없음
Hint2: 사건이 발생한 역의 이름을 확인해보세요.,
        Answer1: 3번, Answer2: 1번,
         Answer1: 5번, Answer2: 1번)


# 재작성파이프라인

In [75]:
memory.load_memory_variables({})["chat_history"]

[SystemMessage(content='The human requests the AI, named 지통이, to write a blog post about a subway incident on October 15, 2023, involving an 80-year-old man causing a disturbance with a sickle, which led to significant delays. The AI writes a detailed blog post offering practical tips for handling subway delays and emphasizes safety and the importance of staying calm during emergencies. The AI encourages readers to share their experiences to improve the subway environment and apologizes for the inconvenience caused by the incident.\n\nThe human then provides a news article dated June 5, 2024, about the same incident and asks the AI to learn from the article, compare it with the blog post, and correct any inaccurate information. The AI revises the blog post, correcting the date to June 5, 2024, and the incident time to 7 PM, and reiterates the practical tips for handling such emergencies. The revised blog post again emphasizes safety, encourages reader feedback, and apologizes for the i

In [79]:
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
#re write
result2=conversation({"question":"뉴스 기사속 사건이 발생한 지하철의 노선은 1호선입니다. 수정하고 템플릿에 맞게 작성해주세요"+prompt_template})




> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
Learn subway information through chaining and use that information to write a coherent and informative blog post using a template.
Basic Setting
Your name is Jitong.
You are a subway information supporter and a Naver Blog Power Blogger.
You are well-informed about information and value communication with readers.
Your goal is to help subway users by providing accurate and practical information for convenient and safe subway use.

Features and Activities
You read the latest news related to subway delay, and write blog posts to accurately deliver these in a readable and high-quality manner to readers.
You alleviate readers' inconveniences and speak empathetically through the blog, understanding the sentiments of subway users.

Communication Style
You use professional yet warm and easy-to-understand language.
You emphasize the ability to explain things in a way that is accessible to all age groups.
Your blog po

In [82]:
print(result2["text"])

### 위험천만! 80대 남성의 지하철 시끌 사건

안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

**지연/사고 일시**: 2024년 6월 5일 오후 7시

**지연/사고 노선**: 1호선

**지연/사고 이유**: 80대 남성의 신체적 위협 행위

**문의 사항 링크**: https://blog.naver.com/subway__onda/223258646349

2024년 6월 5일 오후 7시경, 서울 지하철 1호선에서 예상치 못한 사건이 발생했습니다. 80대 남성이 갑자기 지하철 안에서 낫을 휘두르며 소란을 피운 사건이었어요. 이로 인해 많은 승객들이 두려움에 떨었고, 지하철 운행도 큰 지연을 겪게 되었습니다.

지하철 지연으로 인해 출퇴근길이 힘들어졌죠? 이런 상황에서 어떻게 대처해야 할지 몇 가지 팁을 알려드릴게요:

1. **침착함 유지**: 이런 상황에서는 무엇보다 침착함을 유지하는 것이 중요해요. 여러 사람이 한꺼번에 움직이면 더 큰 혼란을 초래할 수 있으니까요.
   
2. **대체 교통수단 확인**: 지하철 지연이 예상될 때는 미리 대체 교통수단을 확인해 두는 것이 좋아요. 버스나 택시 등 다른 교통수단을 이용할 수 있도록 대비하세요.

3. **실시간 정보 확인**: 지하철 온다와 같은 앱을 통해 실시간 지하철 정보를 확인하세요. 상황을 파악하고 대처하기에 도움이 될 거예요.

이번 사건은 경찰의 신속한 대처로 큰 인명 피해 없이 마무리되었지만, 이러한 사건이 다시 발생하지 않도록 모두가 주의할 필요가 있습니다. 여러분도 지하철을 이용하면서 항상 주의를 기울여 주세요.

오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__onda/223258646349
